# Frontal Gaze classifier

### Dataset autosplit

In [2]:
import tensorflow as tf
import keras
from keras.models import Model
from keras import layers
from keras.layers import Dense
import matplotlib.pyplot as plt

2025-04-16 16:06:52.248493: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-16 16:06:52.262436: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-16 16:06:52.273135: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-16 16:06:52.276147: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-16 16:06:52.291746: I tensorflow/core/platform/cpu_feature_guar

In [5]:
train_ds = keras.preprocessing.image_dataset_from_directory(
    directory='dataset/train',
    labels='inferred',
    label_mode='categorical',
    class_names=['close_look', 'forward_look', 'left_look', 'right_look'],
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=1821,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 14320 files belonging to 4 classes.
Using 11456 files for training.


In [6]:
val_ds = keras.preprocessing.image_dataset_from_directory(
    directory='dataset/train',
    labels='inferred',
    label_mode='categorical',
    class_names=['close_look', 'forward_look', 'left_look', 'right_look'],
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=1821,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 14320 files belonging to 4 classes.
Using 2864 files for validation.


In [7]:
test_ds = keras.preprocessing.image_dataset_from_directory(
    directory='dataset/test',
    labels='inferred',
    label_mode='categorical',
    class_names=['close_look', 'forward_look', 'left_look', 'right_look'],
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=1821,
    validation_split=0,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 40 files belonging to 4 classes.


In [25]:
model = keras.applications.MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
    )

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [26]:
# Create a layer where input is the output of the second last layer
output = Dense(4, activation='softmax', name='predictions')(model.layers[-2].output)

# Then create the corresponding model
model = Model(model.input, output)

In [27]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 112, 112, 32)   │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 112, 112, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 112, 112, 32)   │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 112, 112, 64)   │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 56, 56, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 56, 56, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 56, 56, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 56, 56, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 4,257,868 (16.24 MB)

 Trainable params: 4,235,980 (16.16 MB)

 Non-trainable params: 21,888 (85.50 KB)

In [ ]:
epochs = 37

callbacks = [
    # to save the model after every epoch
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
    # logging
    keras.callbacks.TensorBoard(log_dir="logs", write_graph=True, write_images=False, update_freq="epoch",)
]

model.compile(
    optimizer=keras.optimizers.SGD(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

Epoch 1/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.9877 - loss: 0.0381 - val_accuracy: 0.9721 - val_loss: 0.0783
Epoch 2/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.9899 - loss: 0.0371 - val_accuracy: 0.9804 - val_loss: 0.0647
Epoch 3/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.9894 - loss: 0.0305 - val_accuracy: 0.9759 - val_loss: 0.0773
Epoch 4/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.9945 - loss: 0.0200 - val_accuracy: 0.9850 - val_loss: 0.0515
Epoch 5/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.9935 - loss: 0.0216 - val_accuracy: 0.9801 - val_loss: 0.0693
Epoch 6/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.9948 - loss: 0.0176 - val_accuracy: 0.9825 - val_loss: 0.0558
Epoch 7/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 12s 35ms/step - accuracy: 0.9969 - loss: 0.0138 - val_accuracy: 0.9867 - val_loss: 0.0476
Epoch 8/250
358/358 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - accuracy: 0.9963 - loss: 0

In [20]:
model_trained = keras.models.load_model('./save_at_37.keras')

model_trained.evaluate(
    x=test_ds,
    y=None,
    verbose=True,
    sample_weight=None,
    steps=None,
    callbacks=None,
    return_dict=False,
)

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 475ms/step - accuracy: 0.8646 - loss: 0.4159


[0.3858256936073303, 0.875]

In [ ]:
import numpy as np
img_path = "./dataset/test/close_look/eye_closed (5).png"
img = keras.utils.load_img(img_path, target_size=(224, 224))
img_array = keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
img_array = img_array / 255.0

predictions = model_trained.predict(img_array)
predicted_class = np.argmax(predictions[0])
confidence = np.max(predictions[0])
print(f"{img_path}: {predicted_class} ({confidence:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[[1.0390614e-02 9.8955989e-01 7.7215263e-06 4.1727650e-05]]
./dataset/test/close_look/eye_closed (5).png: 1 (0.99)
